In [33]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import pandas as pd
import os
import pydicom
from tqdm import tqdm
from skimage import transform
import cv2
from PIL import Image
import numpy as np
from pprint import pprint

In [7]:
hypermodel = tf.keras.models.load_model('/kaggle/input/modelh5/model.h5', compile = False)

In [12]:
PATH = "/kaggle/input/rsna-breast-cancer-detection"

# Load the labels
test_df = pd.read_csv(os.path.join(PATH, 'test.csv'))

In [13]:
test_df.head()

,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,prediction_id
0,2,10008,736471439,L,MLO,81,0,21,10008_L
1,2,10008,1591370361,L,CC,81,0,21,10008_L
2,2,10008,68070693,R,MLO,81,0,21,10008_R
3,2,10008,361203119,R,CC,81,0,21,10008_R


In [14]:
!rm -rf /kaggle/working/test_transformed

In [15]:
os.mkdir("/kaggle/working/test_transformed/")

In [25]:
img_width = 224
img_height = 224
img_shape = (img_height, img_width, 1) 

p_id = test_df.patient_id
i_id = test_df.image_id
for pid, iid in tqdm(zip(p_id, i_id)):
    tmpFile = str(pid) + "_" + str(iid) + ".png"
    tmpDCMFile = str(pid) + "/" + str(iid) + ".dcm"
    tmpSrc = "/kaggle/input/rsna-breast-cancer-detection/test_images/" + tmpDCMFile
    tmpDst = "/kaggle/working/test_transformed/" + tmpFile
    imagedata= pydicom.dcmread(tmpSrc)
    img =imagedata.pixel_array
    img = transform.resize(img,img_shape) 
    cv2.imwrite(tmpDst, img)

4it [00:04,  1.16s/it]


In [26]:
testdata_path = "/kaggle/working/test_transformed"
for ii in os.listdir(testdata_path):
    print(testdata_path + "/" + ii)

/kaggle/working/test_transformed/10008_736471439.png
/kaggle/working/test_transformed/10008_1591370361.png
/kaggle/working/test_transformed/10008_361203119.png
/kaggle/working/test_transformed/10008_68070693.png


In [28]:
def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (224, 224, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

In [31]:
testdata_path = "/kaggle/working/test_transformed"
pred_dict = dict()
for ii in os.listdir(testdata_path):
    tmpPath = testdata_path + "/" + ii
    image = load(tmpPath)
    predictions = hypermodel.predict(image)
    score = tf.nn.softmax(predictions[0])
    pred_dict[ii] = float(max(score))
    print(pred_dict[ii])

2023-01-26 11:40:54.372892: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


0.5816646218299866
0.5816646218299866
0.5816646218299866
0.5816646218299866


In [32]:
submission = pd.read_csv("/kaggle/input/rsna-breast-cancer-detection/sample_submission.csv")
submission

,prediction_id,cancer
0,10008_L,0.021168
1,10008_R,0.021168


In [34]:
testD = {"10008_L":{"10008_736471439.png":0, "10008_1591370361.png":0}, 
         "10008_R":{"10008_68070693.png":0,"10008_361203119.png":0}}

for k1 in testD:
    for k2 in testD[k1]:
        testD[k1][k2] = pred_dict[k2]

pprint(testD)

{'10008_L': {'10008_1591370361.png': 0.5816646218299866,
             '10008_736471439.png': 0.5816646218299866},
 '10008_R': {'10008_361203119.png': 0.5816646218299866,
             '10008_68070693.png': 0.5816646218299866}}


In [35]:
for jj in range(submission.shape[0]):
    tmpKey = submission.prediction_id.iloc[jj]
    submission.cancer.iloc[jj] = np.mean(list(testD[tmpKey].values()))

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [36]:
submission.to_csv("submission.csv")
submission

,prediction_id,cancer
0,10008_L,0.581665
1,10008_R,0.581665
